In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import onnx
from onnx_tf.backend import prepare

In [18]:
class Net(nn.Module):    
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [24]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 10, kernel_size=5),
            nn.MaxPool2d(2),
            nn.ReLU(),
            nn.Conv2d(10, 20, kernel_size=5),
            nn.Dropout2d(),
            nn.MaxPool2d(2),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(320, 50),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(50, 10),
            nn.Softmax()
        )
    def forward(self, inputs):
        return self.model(inputs)


In [25]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 1000 == 0:
            print('Train Epoch: {} \tLoss: {:.6f}'.format(
                    epoch,  loss.item()))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the maxlog-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
        

In [26]:
train_loader = torch.utils.data.DataLoader(datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])), batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(datasets.MNIST('../data', train=False, 
                        transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])), batch_size=1000, shuffle=True)

In [27]:
device = torch.device("cuda")
print(device)
model_torch = Net().to(device)
print(model_torch)
optimizer = optim.SGD(model_torch.parameters(), lr=0.01, momentum=0.5)
for epoch in range(10):
    train(model_torch, device, train_loader, optimizer, epoch)
    test(model_torch, device, test_loader) 

cuda
Net(
  (model): Sequential(
    (0): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): ReLU()
    (3): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
    (4): Dropout2d(p=0.5, inplace=False)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): ReLU()
    (7): Flatten(start_dim=1, end_dim=-1)
    (8): Linear(in_features=320, out_features=50, bias=True)
    (9): ReLU()
    (10): Dropout(p=0.5, inplace=False)
    (11): Linear(in_features=50, out_features=10, bias=True)
    (12): Softmax(dim=None)
  )
)
Train Epoch: 0 	Loss: -0.099253


/home/lzw365/.local/lib/python3.8/site-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)



Test set: Average loss: -0.1837, Accuracy: 3251/10000 (33%)

Train Epoch: 1 	Loss: -0.150705

Test set: Average loss: -0.7915, Accuracy: 8050/10000 (80%)

Train Epoch: 2 	Loss: -0.522352


KeyboardInterrupt: 

In [22]:
model_torch.named_modules

<bound method Module.named_modules of Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)>

In [8]:
torch.save(model_torch.state_dict(), 'mnist.pth')

In [9]:
model_torch = Net()
model_torch.load_state_dict(torch.load('mnist.pth'))
dummy_input = Variable(torch.randn(1, 1, 28, 28))
torch.onnx.export(model_torch.eval(), dummy_input, "mnist.onnx")

In [10]:
model_onnx = onnx.load('mnist.onnx')
model_tf = prepare(model_onnx)

2022-04-25 20:39:33.720975: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-25 20:39:33.727853: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-04-25 20:39:33.727875: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-04-25 20:39:33.728301: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN

In [11]:
import numpy as np
from IPython.display import display
from PIL import Image
import matplotlib.pyplot as plt

print('Image 1:')
img = Image.open('two.png').resize((28, 28)).convert('L')
display(img)
temp = np.asarray(img, dtype=np.float32)[np.newaxis, np.newaxis, :, :]
print(temp.shape)
output = model_torch(torch.from_numpy(temp))
print('The digit is classified as', output.detach().numpy(), 'pytorch')
output = model_tf.run(temp)
print('The digit is classified as', output, 'tf')

Image 1:


(1, 1, 28, 28)
The digit is classified as [[-1050.928  -1503.1975 -1432.3577 -1705.7224 -1070.0522 -1097.5026
      0.     -1961.2129 -1064.8805 -1536.6558]] pytorch
The digit is classified as Outputs(_0=array([[-1050.928 , -1503.1974, -1432.3577, -1705.7225, -1070.0521,
        -1097.5027,     0.    , -1961.213 , -1064.8806, -1536.6558]],
      dtype=float32)) tf


In [12]:
print('Image 2:')
img = Image.open('three.png').resize((28, 28)).convert('L')
display(img)
temp = np.asarray(img, dtype=np.float32)[np.newaxis, np.newaxis, :, :]
print(temp.shape)
output = model_torch(torch.from_numpy(temp))
print('The digit is classified as', output.detach().numpy(), 'pytorch')
output = model_tf.run(temp)
print('The digit is classified as', output, 'tf')

Image 2:


(1, 1, 28, 28)
The digit is classified as [[-1674.5397  -1686.7819   -955.34235     0.      -1886.2817  -1205.1274
  -2681.6333  -1189.669    -993.9929   -944.36743]] pytorch
The digit is classified as Outputs(_0=array([[-1674.5398 , -1686.7817 ,  -955.3423 ,     0.     , -1886.2817 ,
        -1205.1274 , -2681.6333 , -1189.6688 ,  -993.9929 ,  -944.36743]],
      dtype=float32)) tf


In [13]:
model_tf.export_graph('mnist.pb')

2022-04-25 20:39:52.598284: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: mnist.pb/assets


INFO:tensorflow:Assets written to: mnist.pb/assets


In [14]:
for param in model_torch.parameters():
    print(param.data)
    print(param.data.shape)

tensor([[[[-0.3833,  0.0220, -0.0965,  0.1975,  0.2135],
          [-0.3189, -0.1862,  0.0243,  0.4145,  0.2126],
          [-0.1882,  0.1057,  0.3692,  0.3993,  0.1196],
          [ 0.1749,  0.1542,  0.2534,  0.0729,  0.0680],
          [ 0.1706,  0.0412, -0.1490,  0.1393, -0.1932]]],


        [[[-0.0095, -0.1795,  0.0275, -0.1768, -0.1461],
          [ 0.3106, -0.2124, -0.1373,  0.1904, -0.0599],
          [ 0.3169,  0.0625, -0.1395,  0.4066,  0.3856],
          [ 0.1472,  0.1810, -0.0762,  0.0525,  0.2633],
          [ 0.0116,  0.0064, -0.3337, -0.2542, -0.0572]]],


        [[[-0.0179,  0.1203,  0.0657,  0.2108, -0.0585],
          [ 0.1656,  0.0551, -0.0301,  0.2053, -0.0580],
          [-0.0362,  0.0716,  0.2244,  0.0163, -0.1420],
          [ 0.2148, -0.0888,  0.1408, -0.0614, -0.2482],
          [ 0.2197,  0.0198, -0.1216, -0.2027, -0.1967]]],


        [[[ 0.2603,  0.0345, -0.0561, -0.1583, -0.2309],
          [ 0.1057,  0.2851, -0.2063, -0.1851, -0.2932],
          [ 0.3078,

In [15]:
import tensorflow as tf
imported = tf.saved_model.load('mnist.pb')
f = imported.signatures["serving_default"]
#print(f(x=tf.constant([[1.]])))

In [16]:
f(tf.convert_to_tensor(temp))

{'20': <tf.Tensor: shape=(1, 10), dtype=float32, numpy=
 array([[-1674.5398 , -1686.7817 ,  -955.3423 ,     0.     , -1886.2817 ,
         -1205.1274 , -2681.6333 , -1189.6688 ,  -993.9929 ,  -944.36743]],
       dtype=float32)>}

In [17]:
model_new = tf.keras.models.load_model('mnist.pb')
f_new = model_new.signatures["serving_default"]

In [23]:
import torch
from torch import nn
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [24]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [25]:
x = torch.randn(1, 10)

class MyModel(nn.Module):
    def __init__(self, inplace=False):
        super(MyModel, self).__init__()
        self.fc = nn.Linear(10, 10)
        self.fc1 = nn.Linear(10, 1)
        self.relu = nn.ReLU(inplace=inplace)
        
    def forward(self, x):
        x = self.relu(self.fc1(self.fc(x)))
        return x

model = MyModel()

In [28]:
model.eval()
# out of place
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook
d = dict(model.named_modules())

for i in d:
    print(i)
    model.fc1.register_forward_hook(get_activation(i))

model.fc.register_forward_hook(get_activation('fc'))
output = model(x)
activation['fc']


fc
fc1
relu


tensor([[0.2324]])

In [33]:
model.eval()
hooks = {}
def get_activation(name):
    def hook(model, input, output):
        hooks[name] = output.detach()
    return hook


for name, module in model.named_modules():
    print(name)
    hooks[name] = module.register_forward_hook(get_activation(name))
    output = model(x)
    print(hooks[name])
lists = model.named_modules()
print(lists)



tensor([[0.2324]])
fc
tensor([[ 0.3466,  0.5461,  0.1191, -0.3720, -0.6417,  0.6475,  0.3437, -0.1969,
         -0.3114, -1.3142]])
fc1
tensor([[0.2324]])
relu
tensor([[0.2324]])
<generator object Module.named_modules at 0x7f5b2b3ccc80>


In [ ]:
output = model(x)
print(hooks['fc'])
output = model(x)
print(hooks['fc1'])

In [34]:
class Net(nn.Module):    
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [42]:
x1 = torch.randn(1, 28, 28).to(device)
model = Net().to(device)
print(model)

Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)


In [43]:
model.eval()
hooks = {}
def get_activation(name):
    def hook(model, input, output):
        hooks[name] = output.detach()
    return hook


for name, module in model.named_modules():
    print(name)
    hooks[name] = module.register_forward_hook(get_activation(name))
    output = model(x1)
    print(hooks[name])
lists = model.named_modules()
print(lists)


tensor([[-2.1330, -2.4107, -2.3183, -2.2274, -2.2931, -2.4593, -2.1553, -2.4574,
         -2.3949, -2.2410]], device='cuda:0')
conv1
tensor([[[ 0.3552, -0.3835, -0.3798,  ...,  0.6531, -0.3819,  0.9250],
         [-0.3931,  0.4347, -0.8004,  ...,  1.0041,  0.2548,  0.4114],
         [-0.1936, -0.5977,  0.3851,  ..., -1.1718, -0.1572,  0.4118],
         ...,
         [-0.4834,  0.4055,  0.8806,  ..., -0.4012, -0.4326, -0.0167],
         [-0.1985, -1.4211, -0.9980,  ...,  0.7819,  0.6955, -0.2902],
         [-0.3093, -0.8619, -0.4449,  ...,  0.1748,  0.0424, -1.1426]],

        [[-0.0847,  0.2656,  0.2576,  ..., -0.0345,  0.3506,  0.3423],
         [ 1.1895,  0.7303,  0.1480,  ...,  0.1669, -0.4908, -0.0397],
         [-1.1355,  0.1719, -0.3975,  ...,  0.9463,  0.8590,  1.0641],
         ...,
         [ 0.7148,  1.6135,  0.8700,  ..., -0.7308,  0.3505, -1.2597],
         [-0.3741,  0.4877,  0.0056,  ...,  0.5539, -0.4294,  0.9344],
         [ 0.6951, -0.0517,  0.2878,  ...,  0.4206, -0.

In [14]:
import torch
import torch.nn as nn
# the VGG11 architecture
class VGG11(nn.Module):
    def __init__(self, in_channels=1, num_classes=10):
        super(VGG11, self).__init__()
        self.in_channels = in_channels
        self.num_classes = num_classes
        # convolutional layers 
        self.conv_layers = nn.Sequential(
            nn.Conv2d(self.in_channels, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        # fully connected linear layers
        self.linear_layers = nn.Sequential(
            nn.Linear(in_features=512, out_features=4096),
            nn.ReLU(),
            nn.Dropout2d(0.5),
            nn.Linear(in_features=4096, out_features=4096),
            nn.ReLU(),
            nn.Dropout2d(0.5),
            nn.Linear(in_features=4096, out_features=self.num_classes)
        )
    def forward(self, x):
        x = self.conv_layers(x)
        # flatten to prepare for the fully connected layers
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

In [15]:
model = VGG11()
model

VGG11(
  (conv_layers): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU()
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU()
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU()
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_layers): Sequential(
    (0): Linear(in_features=512, 